# simple prompt

In [ ]:
# load model
from langchain.chat_models import init_chat_model
model = init_chat_model("qwen2.5:latest", model_provider="ollama")

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage
def callBack(text):
  messages = [
      SystemMessage("You are a helpful assistant and answer the question as best as you can."),
      HumanMessage(text),
  ]
  return model.invoke(messages).content
  
callBack("tell me a joke")

# classification

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models import init_chat_model
from pydantic import BaseModel, Field

tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)


class Classification(BaseModel):
    sentiment: str = Field(description="The sentiment of the text")
    aggressiveness: int = Field(
        description="How aggressive the text is on a scale from 1 to 10"
    )
    language: str = Field(description="The language the text is written in")


# LLM
llm = init_chat_model("qwen2.5:latest", model_provider="ollama").with_structured_output(
    Classification
)


inp = "I feel very bad for missing the train."
prompt = tagging_prompt.invoke({"input": inp})
response = llm.invoke(prompt)

response

response.model_dump() # dict output

# Tool Calling

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.chat_models import init_chat_model
import requests
import random

@tool
def multiply(x: float, y: float) -> float:
    """Multiply 'x' times 'y'."""
    return x * y

@tool
def exponentiate(x: float, y: float) -> float:
    """Raise 'x' to the 'y'."""
    return x**y

@tool
def add(x: float, y: float) -> float:
    """Add 'x' and 'y'."""
    return x + y

@tool
def get_weather(city: str = "New York") -> str:
    """Fetch the weather report for a given 'city'."""
    return f"The current temperature in {city} is 15°C with mild snow fall."

@tool
def get_traffic_report(city: str) -> str:
    """Fetch the current traffic report for a given 'city'."""
    return f"Traffic in {city}: Moderate traffic with occasional congestion."

# List of tools including the new weather tool
tools = [multiply, exponentiate, add, get_weather, get_traffic_report]

# Define the prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You're a helpful assistant"), 
    ("human", "{input}"), 
    ("placeholder", "{agent_scratchpad}"),		
])

# Initialize the LLM
llm = init_chat_model("qwen2.5:latest", model_provider="ollama")

# Create the agent with the tools and prompt
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Step 4: Invoke the tool with a query including the weather request
result = agent_executor.invoke({
    "input": "How's the traffic in New York?",
})

# Step 5: Print the result
print(result)


# News Scriptwriter

In [ ]:
# load model
from langchain.chat_models import init_chat_model
model = init_chat_model("qwen2.5:latest", model_provider="ollama")

from langchain_core.messages import HumanMessage, SystemMessage
def callBack(text):
  messages = [
      SystemMessage("You are a helpful smart news scriptwriter capable of intrepreting news from internet scrapped news article."),
      HumanMessage(text),
  ]
  return model.invoke(messages).content
  
callBack("tell me a joke")

# Junior News Scriptwriter

In [ ]:
# load model
from langchain.chat_models import init_chat_model
llm = init_chat_model("qwen2.5:latest", model_provider="ollama")

from typing import Optional
from pydantic import BaseModel, Field

class Article(BaseModel):
    """You are a intelligent AI, capable of identifying the genre of a news article."""
    
    title: str = Field(description="Title of the news article")
    genre: str = Field(description="Genre of the news article")
    rating: Optional[int] = Field(
        default=None, description="How important the news article is, from 1 to 10"
    )
    
llm = llm.with_structured_output(Article)

In [ ]:
def callBack(text):
  return llm.invoke(text)
  
callBack("tell me a joke")

In [ ]:
https://python.langchain.com/docs/how_to/structured_output/#the-with_structured_output-method

# RAG

## Ingestion Pipeline

In [ ]:
# llm
from langchain.chat_models import init_chat_model
llm = init_chat_model("qwen2.5:latest", model_provider="ollama")

# embeddings
from langchain_ollama import OllamaEmbeddings
embeddings = OllamaEmbeddings(model="mxbai-embed-large:latest")

# vector db - create
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

embedding_dim = len(embeddings.embed_query("hello world"))
index = faiss.IndexFlatL2(embedding_dim)

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

# vector db - write
import os
import re
from uuid import uuid4
from langchain_core.documents import Document

directory = "./tmpd/text/"

def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)

documents = []
for file in sorted_alphanumeric(os.listdir("./tmpd/text/")):
  full_path = os.path.join(directory, file)
  print(full_path)
  with open(full_path, 'r') as f:
    documents.append(
    	Document(page_content=f.read(), metadata={"source": "jokes"},)
    	)
	
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

# vector db - save
vector_store.save_local("qwen_knowledge")

## Retrieval Pipeline

In [ ]:
# llm
from langchain.chat_models import init_chat_model
llm = init_chat_model("qwen2.5:latest", model_provider="ollama")

# embeddings
from langchain_ollama import OllamaEmbeddings
embeddings = OllamaEmbeddings(model="mxbai-embed-large:latest")

# load vector db
from langchain_community.vectorstores import FAISS
vector_store = FAISS.load_local(
    "qwen_knowledge", embeddings, allow_dangerous_deserialization=True
)

from langchain import hub
from langchain_core.documents import Document
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

# Define prompt for question-answering
prompt = hub.pull("rlm/rag-prompt")

# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}

def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}

# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

response = graph.invoke({"question": "What is Task Decomposition?"})
print(response["answer"])

## Ingestion and Retrieval

In [ ]:
# llm
from langchain.chat_models import init_chat_model
llm = init_chat_model("qwen2.5:latest", model_provider="ollama")

# embedding
from langchain_ollama import OllamaEmbeddings
embeddings = OllamaEmbeddings(model="mxbai-embed-large:latest")

import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

embedding_dim = len(embeddings.embed_query("hello world"))
index = faiss.IndexFlatL2(embedding_dim)

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)


import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

# Load and chunk contents of the blog
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

# Index chunks
_ = vector_store.add_documents(documents=all_splits)

# Define prompt for question-answering
prompt = hub.pull("rlm/rag-prompt")


# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()


response = graph.invoke({"question": "What is Task Decomposition?"})
print(response["answer"])

In [ ]:
# https://python.langchain.com/docs/tutorials/rag/

In [ ]:
# https://python.langchain.com/docs/integrations/vectorstores/faiss/#saving-and-loading